# How do I _copy_ a file from another one of _my projects_?
### Overview
Files can be copied from one of two sources:
1. other projects you are a member of
2. public reference.  

Here we focus on copying a file from _your own project_. 

### Prerequisites
 1. You need to be a member (or owner) of _at least one_ project.
 2. You need your _authentication token_ and the API needs to know about it. See <a href="Setup_API_environment.ipynb">**Setup_API_environment.ipynb**</a> for details.
 3. You understand how to <a href="projects_listAll.ipynb" target="_blank">list</a> projects you are a member of (we will just use that call directly and pick one here).
 4. You understand how to <a href="files_listAll.ipynb" target="_blank">list</a> files within one of your projects (we will just use that call directly here).

### Warning
You are only permitted to copy a file from a `controlled` project to a _target project_ if that project is also a `controlled` project.

## Imports
We import the _Api_ class from the official sevenbridges-python bindings below.

In [ ]:
import sevenbridges as sbg

## Initialize the object
The `Api` object needs to know your **auth\_token** and the correct path. Here we assume you are using the credentials file in your home directory. For other options see <a href="Setup_API_environment.ipynb">Setup_API_environment.ipynb</a>

In [ ]:
# [USER INPUT] specify credentials file profile {cgc, sbg, default}
prof = 'default'

config_file = sbg.Config(profile=prof)
api = sbg.Api(config=config_file)

## Copy a file from another project
We will first find our _source\_project_ and _my\_project_ (target), then list the files within the source project, and copy a file from **_source\_project_ -> _destination\_project_**.

The critical information for this POST is the **file_id**. Note, you are allowed to copy the same file as many times as you like. However, duplicates will be automatically have a prefix attached of (\_1\_, \_2\_, etc) depending on how many times you copy the file.

To make these results very obvious, use an empty project as your destination\_project (e.g. your <a href=projects_makeNew.ipynb> cookbook example project</a>) or change the _name_ in the _data_ dictionary to something like 'Dept of Awesome.copy'

---
**NOTE**
The file listing method does not list files in subfolders, only the files and folders in root of the project are listed

---

In [ ]:
# [USER INPUT] Set source project id, destination project id and file index here:
source_project_id = 'username/source-project'        
destination_project_id = 'username/destination-project'
f_index =  # file to copy, here we take the first file from the source project


# Find TARGET and YOUR projects
source_project = api.projects.get(source_project_id)
destination_project = api.projects.get(destination_project_id)

# LIST all files in the source project and pick the first one
source_files = api.files.query(limit=100, project=source_project)
source_file = source_files[f_index]

# LIST all files in the destination project. We add the names parameter to only list file with the source file name
destination_files = list(api.files.query(limit=100, project=destination_project, names=[source_file.name]))

# Check if first file already exists in the target project
if destination_files:
    print('File already exists in second project, skipping.')
else:
    print('File {} does not exist in project {}; copying now.'.format(
            source_file_names[f_index], destination_project.id))
    
    
    my_new_file = source_file.copy(project=destination_project.id,
                                   name=source_file.name)

    # re-list files in target project to verify the copy worked
    my_files = [f.name for f in api.files.query(
            limit=100, project=destination_project.id).all()]
    
    if source_file.name in my_files:
        print('Sucessfully copied one file!')
    else:
        print('Something went wrong...')

## Additional Information
Detailed documentation of this particular REST architectural style request is available [here](http://docs.sevenbridges.com/docs/copy-a-file)